In [ ]:
from PIL import Image
import os
import rsa

def mensagem_para_bits(mensagem):
    # Converte a mensagem para uma string de bits
    bits = ''.join([format(ord(char), '08b') for char in mensagem])
    return bits

def ocultar_mensagem(imagem_caminho, mensagem):
    imagem = Image.open(imagem_caminho)
    imagem = imagem.convert("RGB")
    pixels = imagem.load()

    mensagem += "###"  # Sinal de final da mensagem
    bits_mensagem = mensagem_para_bits(mensagem)
    bits_index = 0

    for y in range(imagem.height):
        for x in range(imagem.width):
            if bits_index < len(bits_mensagem):
                r, g, b = pixels[x, y]
                r = (r & ~1) | int(bits_mensagem[bits_index])  # Modifica o bit menos significativo de r
                bits_index += 1
                if bits_index < len(bits_mensagem):
                    g = (g & ~1) | int(bits_mensagem[bits_index])  # Modifica o bit menos significativo de g
                    bits_index += 1
                if bits_index < len(bits_mensagem):
                    b = (b & ~1) | int(bits_mensagem[bits_index])  # Modifica o bit menos significativo de b
                    bits_index += 1
                pixels[x, y] = (r, g, b)
            else:
                directory, filename = os.path.split(imagem_caminho)
                name, extension = os.path.splitext(filename)
                text_to_insert = '_esteganografia'

                new_filename = f"{name}{text_to_insert}{extension}"
                output_path = os.path.join(directory, new_filename)

                imagem.save(output_path)
                return output_path


def revelar_mensagem(imagem_caminho):
    imagem = Image.open(imagem_caminho)
    imagem = imagem.convert("RGB")
    pixels = imagem.load()

    bits_mensagem = ""
    for y in range(imagem.height):
        for x in range(imagem.width):
            r, g, b = pixels[x, y]
            bits_mensagem += str(r & 1)
            bits_mensagem += str(g & 1)
            bits_mensagem += str(b & 1)

    bytes_mensagem = [bits_mensagem[i:i+8] for i in range(0, len(bits_mensagem), 8)]
    mensagem = ''.join([chr(int(byte, 2)) for byte in bytes_mensagem])

    return mensagem.split("###")[0]  # Remove o sinal de final de mensagem


In [ ]:
def gerar_chaves():
    (chave_publica, chave_privada) = rsa.newkeys(512)
    return chave_publica, chave_privada

# Função para criptografar uma mensagem usando a chave pública
def criptografar_mensagem(mensagem, chave_publica):
    mensagem_bytes = mensagem.encode('utf-8')
    mensagem_encriptada = rsa.encrypt(mensagem_bytes, chave_publica)
    return mensagem_encriptada

# Função para decriptar a mensagem usando a chave privada
def descriptografar_mensagem(mensagem_encriptada, chave_privada):
    mensagem_decriptada = rsa.decrypt(mensagem_encriptada, chave_privada)
    return mensagem_decriptada.decode('utf-8')

In [ ]:
import hashlib

def gerar_hash(filename):
  sha256_hash = hashlib.sha256()
  with open(filename, "rb") as f:
    for byte_block in iter(lambda: f.read(4096),b""):
      sha256_hash.update(byte_block)
    return sha256_hash.hexdigest()

In [ ]:
def menu():

    chave_publica = None
    chave_privada = None
    mensagem_encriptada = None


    while True:
        print("\n=== MENU ===")
        print("1. Escrever em uma imagem usando steganografia")
        print("2. Recuperar o texto de uma imagem")
        print("3. Gerar Hash das duas imagens")
        print("4. Criptografa a mensagem original")
        print("5. Decriptar a mensagem")
        print("S. Sair")

        opcao = input("Escolha uma opção: ")

        if opcao == '1':
            print("Embutir texto em uma imagem usando esteganografia")
            imagem = input("Caminho da imagem: ")
            mensagem = input("Mensage: ")
            imagem_mensagem = ocultar_mensagem(imagem, mensagem)
            print("Imagem gerada no caminho: " + imagem_mensagem)
            # Aqui você pode chamar uma função ou executar um código para a Opção 1
        elif opcao == '2':
            print("recuperar o texto inserido em uma imagem alterada ")
            if imagem_mensagem != '':
                mensagem_escondida = revelar_mensagem(imagem_mensagem)
                print("Mensagem: " + mensagem_escondida)
            else:
                print("Nenhuma imagem foi inserida")

        elif opcao == '3':
            print("\n--------Hash das imagens--------")
            imagem_1 = input("Caminho da imagem 1: ")
            imagem_2 = input("Caminho da imagem 2: ")
            hash1 = gerar_hash(imagem_1)
            hash2 = gerar_hash(imagem_2)

            print("Hash da imagem 1: " + hash1)
            print("Hash da imagem 2: " + hash2)

        elif opcao == '4':
            print("Criptografar a mensagem original")
            mensagem = input("Digite a mensagem a ser criptografada: ")
            chave_publica, chave_privada = gerar_chaves()
            mensagem_encriptada = criptografar_mensagem(mensagem, chave_publica)
            print("Mensagem criptografada com sucesso.")
            # Opcional: armazenar a mensagem criptografada em uma variável para inserção em uma imagem na opção (1)
            print('')

        elif opcao == '5':
            print("Decriptar a mensagem")
            if mensagem_encriptada and chave_privada:
                try:
                    mensagem_decriptada = descriptografar_mensagem(mensagem_encriptada, chave_privada)
                    print("Mensagem decriptada: " + mensagem_decriptada)
                except rsa.DecryptionError:
                    print("Erro ao decriptar a mensagem. Verifique se a chave privada está correta.")
            else:
                print("Nenhuma mensagem criptografada ou chave privada disponível.")
            print('')

        elif opcao.upper() == 'S':
            print("Saindo do programa...")
            break
        else:
            print("Opção inválida. Tente novamente.")

menu()


=== MENU ===
1. Escrever em uma imagem usando steganografia
2. Recuperar o texto de uma imagem
3. Gerar Hash das duas imagens
4. Criptografa a mensagem original
5. Decriptar a mensagem
S. Sair
